In [23]:
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

In [24]:
def print_matrix(M: np.ndarray):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    """
    for row in M:
        print(f"{np.array2string(row, precision=2 ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ')}")
    print('\n')

def print_data_ext_ps(V1: np.complex_, V2: np.complex_):
    """
    Function to print data on the effect of the external phaseshift
    that's used to match the phases of two given elements.
    
    Parameters
    ------
    V1 : element of the auxillary matrix `V`
    V2 : subsequent element of **V1**
    
    ---
    
    Further description:
    ---
    When it comes to the effect of the external phaseshift  `P`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x,y+1]``
    
    ---
    When it comes to the effect of `exp(i*summa)` in `M`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x-1,y-1]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x+1,y+1]`` ,  ``V2 = V[x,y+1]``
    
    """
    
    eq_stat = (np.angle(V1) == np.angle(V2))
    eq_str = "\nThey are matching!"
    neq_str = "\nThey are NOT matching!\n"

    print('Affected elements:\t',
        "{:.2f}, {:.2f}".format(V1, V2))
    print('Corresponding angles:\t',
        "{:.2f}, {:.2f}".format(np.angle(V1), np.angle(V2)))

    if eq_stat:
        print(eq_str)
    else:
        print(neq_str)
        print('Full length of angles:\n',
            "{}, {}".format(np.angle(V1), np.angle(V2)))
        print('\nTheir difference:\t',
            "{}".format(np.angle(V1) - np.angle(V2)))

In [25]:
U = itf.random_unitary(5)

# Odd diagonals

### Finding external phaseshifts
They match the given elements' phases

In [26]:
V = np.conjugate(U.T)
# odd diags: 1,3,5...
m = U.shape[0]
j = 1

x = m-1
y = j-1

P = itf.external_ps(m, j, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])

Affected elements:	 0.11-0.30j, 0.12-0.33j
Corresponding angles:	 -1.22, -1.22

They are matching!


## Finding internal phaseshifts
$\delta$ and $\sum$

In [27]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = 0

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	-0.82-0.00j	 -> real: False
angle:	-180.0°

Block of M:

[-0.73-1.28e-17j,  0.68-1.38e-17j]
[0.68-1.38e-17j, 0.73+1.28e-17j]




In [28]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(0.00) Im(-0.00)
abs: 0.00, angle: -76.25°



## Even diagonals

### Finding external phaseshifts
They match the given elements' phases

In [29]:
# V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0

P = itf.external_ps(m, j, V[x,y], V[x-1,y])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(P,V)
# print("NOTE: {}. row was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

Affected elements:	 -0.12+0.51j, -0.06+0.26j
Corresponding angles:	 1.80, 1.80

They are NOT matching!

Full length of angles:
 1.7964390042096705, 1.796439004209671

Their difference:	 -4.440892098500626e-16


## Finding internal phaseshifts
$\delta$ and $\sum$

In [30]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(-V[x-1,y], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = np.angle(V[x+1,y+1]) - np.angle(V[x-1,y+1]*np.cos(delta) - V[x,y+1]*np.sin(delta))

# print('\nangles to get equal by summa:\n',
#       np.angle(V[x+1,y+1]),
#       np.angle(V[x,y+1]*np.cos(delta) - V[x+1,y+1]*np.sin(delta)),'\n',
#       'summa:',summ)

# summ = 0

# might need to change into 'k' dependence
modes = [x-1, x]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])
# # print_matrix(M)


# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.48+0.00j	 -> real: False
angle:	2.0742295333050472e-14°


In [31]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
print('Elements affected by e^(i*summ):\n')
print_data_ext_ps(V[x+1,y+1], V[x,y+1])

Nulled element:	Re(-0.00) Im(-0.00)
abs: 0.00, angle: -167.26°

Elements affected by e^(i*summ):

Affected elements:	 0.16-0.44j, 0.24-0.65j
Corresponding angles:	 -1.22, -1.22

They are NOT matching!

Full length of angles:
 -1.2238644580782663, -1.2238644580782665

Their difference:	 2.220446049250313e-16


### Continuation on the chosen diagonal
Without correct $\sum$ it's wrong

In [32]:
x += 1
y += 1

delta = itf.custom_arctan(-V[x-1,y], V[x,y])
print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = 0

modes = [x-1, x]     # initial mode-pairs
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ)
V = np.matmul(M,V)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])

# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(x,x+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.97-0.00j	 -> real: False
angle:	-4.147034211566518e-15°


In [33]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(0.16) Im(-0.44)
abs: 0.47, angle: -70.12°

